# TSNE

In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
from sklearn import preprocessing
import torchvision.models as models
#from torchsummary import summary
import torch.nn as nn
from src.models import WhaleDoModel
from sklearn.model_selection import StratifiedKFold, train_test_split
from src.utils import *
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import torch
#import imutils
import tensorflow as tf
from src.dataloader import WhaleDoDataset
import pickle as pkl
from pytorch_metric_learning import losses, miners
from torch.utils.data import DataLoader
from src.config import get_config

## Get train_df

In [2]:
config = get_config()
# load and parse the dataframe and get the label encoder as well
train_df, label_encoder = load_csv_and_parse_dataframe(config['csv_name'], root_dir=config['root_dir'], drop_columns=['timestamp', 'encounter_id'])
# get the average height and width of the dataset (rotate if viewpoint -1 or 1 (left / right))
config['dataset']['height'], config['dataset']['width'] = get_avg_height_width(train_df)
# get the mean and std of the dataset
config['dataset']['mean'], config['dataset']['std'] = get_mean_and_std_of_dataset(train_df)
# create the dataset objects
train_data = WhaleDoDataset(train_df, config, mode='train')

train_df = train_df.reset_index()
train_df.head()

,image_id,path,height,width,viewpoint,date,whale_id
0,train0000,data/images/train0000.jpg,463,150,0,2017-08-07,0
1,train0001,data/images/train0001.jpg,192,81,0,2019-08-05,1
2,train0002,data/images/train0002.jpg,625,183,0,2017-08-07,2
3,train0003,data/images/train0003.jpg,673,237,0,2017-08-07,3
4,train0004,data/images/train0004.jpg,461,166,0,2018-08-10,4


## Get three samples from train_df
In this part a row from train_df is randomly sampled (the anchor). If there are more whale_id's of this sample in train_df, the program will select randomly another row with the same whale_id (the positive). After that it will select a row which does not have this whale_id (the negative).

In [3]:
# cell to get the samples from the train_df
matched = False

while matched == False:
    random_sample = train_df.sample()
    index = random_sample.index[0]
    random_sample_id = int(random_sample['whale_id'])
    random_sample_image_id = random_sample['image_id'].tolist()[0]
    
    positive_df = train_df.loc[train_df['whale_id']== random_sample_id]
    positive_df = positive_df[positive_df.image_id != random_sample_image_id]
    row_count = len(positive_df.head())

    
    if row_count > 1:
        positive = positive_df.sample()
        index_positive = positive.index[0]
        negative =  train_df.loc[train_df['whale_id'] != random_sample_id].sample()
        index_neg = negative.index[0]
        matched = True

In [4]:
random_sample.head()

,image_id,path,height,width,viewpoint,date,whale_id
3760,train3760,data/images/train3760.jpg,450,126,0,2018-08-09,529


In [5]:
positive.head()

,image_id,path,height,width,viewpoint,date,whale_id
5587,train5587,data/images/train5587.jpg,551,177,0,2018-08-09,529


In [6]:
negative.head()

,image_id,path,height,width,viewpoint,date,whale_id
4756,train4756,data/images/train4756.jpg,217,100,0,2019-08-29,96


In [7]:
print('indexes are: ', index, index_positive, index_neg)

indexes are:  3760 5587 4756


## Use __getitem__ and make a model

In [8]:
# get the three items
anchor_item = train_data.__getitem__(index)
positive_item = train_data.__getitem__(index_positive)
negative_item = train_data.__getitem__(index_neg)

In [9]:
# print(anchor_item)
# print(positive_item)
# print(negative_item)

In [10]:
# from old code
# init loss function and a miner. The miner samples for training samples
loss_func = losses.TripletMarginLoss(margin=config['margin'])
miner = miners.TripletMarginMiner(margin=config['margin'])

device = config['device']
# init model and optimizer
model = WhaleDoModel(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/appuser/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

/srv/conda/envs/condaenv/lib/python3.9/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


WhaleDoModel(
  (backbone): BackBone(
    (model): ResNet(
      (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inp

## Make x_batch and y_batch

In [14]:
# make the three items into tensors and concatenate them into x_batch and y_batch
anchor_x, anchor_y = anchor_item['image'], anchor_item['label']
positive_x, positive_y = positive_item['image'], positive_item['label']
negative_x, negative_y = negative_item['image'], negative_item['label']

# torch.cat is to concatenate along existing dimensions
# x_batch = torch.cat((anchor_x,positive_x,negative_x),1).to(config['device'])
# stack is to create a new dimension (batches) for the batches. You could also manually create a singleton dimension
# in the first position and then use torch.cat(..., 0)
x_batch = torch.stack((anchor_x,positive_x,negative_x),0).to(config['device'])
y_batch = torch.tensor((anchor_y,positive_y,negative_y)).to(config['device'])

print(x_batch.shape)
print(y_batch.shape)

torch.Size([3, 4, 561, 173])
torch.Size([3])


## Get the Embeddings (ERROR)
This part doesn't work. It expected 4 dimensional input and it received 3 dimensional. Since dataloader isn't used, I don't think the augmentations are done and it thus is 3 dimensional. (i think)

In [15]:
#set the gradients to zero
optimizer.zero_grad()

#compute embeddings
embeddings = model(x_batch)

#mine for hard pairs
# mined_pairs = miner(embeddings, y_batch)
#compute loss

# Not sure if you need the loss either 

# loss = loss_func(embeddings, y_batch)#, mined_pairs)


# You don't need to calculate gradients or perform backprop cause you're just testing the model. use model.eval() or something similar.

#calculate gradients
# loss.backward()
#update weights
# optimizer.step()


# Old Playground

In [ ]:
# config = get_config()
# # load and parse the dataframe and get the label encoder as well
# train_df, label_encoder = load_csv_and_parse_dataframe(config['csv_name'], root_dir=config['root_dir'], drop_columns=['timestamp', 'encounter_id'])
# # get the average height and width of the dataset (rotate if viewpoint -1 or 1 (left / right))
# config['dataset']['height'], config['dataset']['width'] = get_avg_height_width(train_df)
# # get the mean and std of the dataset
# config['dataset']['mean'], config['dataset']['std'] = get_mean_and_std_of_dataset(train_df)


# # create the dataset objects
# train_data = WhaleDoDataset(train_df, config, mode='train')

# #### sample from train_df the right index and use it into get_item , two of same label ####
# print(train_data.__getitem__(100)) ##### set __getitem__ to false #####

# # create dataloaders
# train_loader = DataLoader(train_data, config['batch_size'], shuffle=True)

# # init loss function and a miner. The miner samples for training samples
# loss_func = losses.TripletMarginLoss(margin=config['margin'])
# miner = miners.TripletMarginMiner(margin=config['margin'])


# device = config['device']
# # init model and optimizer
# model = WhaleDoModel(config)
# optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
# model.to(device)

In [ ]:
# for i, batch in enumerate(train_loader):

    
# ##### MAKE THE TRIPKETS INTO X_BATCH #####
#     #move tensors to device (gpu or cpu)
#     x_batch, y_batch = batch['image'].to(config['device']), batch['label'].to(config['device'])

#     #set the gradients to zero
#     optimizer.zero_grad()

#     #compute embeddings
#     embeddings = model(x_batch)

#     #mine for hard pairs
#     # mined_pairs = miner(embeddings, y_batch)
#     #compute loss
#     loss = loss_func(embeddings, y_batch)#, mined_pairs)

#     #calculate gradients
#     loss.backward()
#     #update weights
#     optimizer.step()

#     break

In [ ]:
# for i, (x_batch, y_batch) in enumerate(train_loader):
#     optimizer.zero_grad()
#     embeddings = model(x_batch)
#     loss = loss_func(embeddings, y_batch)
#     loss.backward()
#     optimizer.step()

#     print(i)
#     break